In [1]:
from data_scraping import fetch_stat_details

In [ ]:
#Example Usage

STAT_ID = "102"             # Scoring Average
YEAR    = 2024              # seasonal stats
TOURNEY = None              # set to string to filter to a single event

# fetch the DataFrame
df = fetch_stat_details(STAT_ID, year=YEAR, tournament_id=TOURNEY)

print(f"\nTop 5 for stat {STAT_ID} / year {YEAR} / tourney {TOURNEY}:")
print(df.head())


Top 5 for stat 102 / year 2024 / tourney None:
   rank playerId     playerName       % Fairways Hit Possible Fairways
0     1    46414      Aaron Rai  72.02%          857             1,190
1     2    36884     Ben Kohles  70.57%          832             1,179
2     3    29535  Brice Garnett  70.51%          471               668
3     4    52513   Carson Young  70.50%          839             1,190
4     5    40026  Daniel Berger  70.42%          745             1,058


In [8]:
#Create a Dictionary for relevant stats and their codes

stats = {
    #Strokes Gained
    'SG: Total': '02675', 
    'SG: Off the Tee' : '02567',
    'SG: Tee to Green' : '02674',
    'SG: Approach the Green' : '02568', 
    'SG: Around the Green' : '02569',

    #Off the tee
    'Total Driving' : '129',
    'Driving Distance' : '101', 
    'Driving Accuracy Percentage' : '102',
    'Ball Speed' : '02402',

    #Approaching the Green
    'Ball Striking' : '158',
    'Greens or Fringe in Regulation' : '02437',
    'Proximity to Hole' : '331', 
    'GIR Percentage from Other than Fairway' : '199',
    'Going for the Green' : '419',
    'Going for the Green - Hit Green Pct' : '486', 

    #Around the Green
    'Scrambling' : '130', 
    'Proximity to Hole (ARG)' : '374',
    'Sand Save Percentage' : '111', 
    'Scrambling from Rough' : '363',
    'Scrambling from >30 YDS' : '366',

    #Putting
    'Putting Average' : '104', 
    'Putts per Round' : '119', 
    'One-Putt Percentage' : '413',
    '3-Putt Avoidance' : '426', 
    'Putting Inside 10ft' : '484',
    'Putts Made per Event Over 10ft' : '434',
    
    #Scoring
    'Scoring Average':'120',
    'Birdie Average': '156',
    'Birdie or Better Percentage' : '352',
    'Bogey Avoidance' : '02414',
    'Par 3 Scoring Avg' : '142', 
    'Par 4 Scoring Avg' : '143', 
    'Par 5 Scoring Avg' : '144',
    'Scoring Differential Field Avg' : '02417'
}

In [15]:
# ─── choose your range of years ─────────────────────────────────────────
YEARS = range(2010, 2025) 

# ─── container: stat_dfs[year][stat_name] → DataFrame ────────────────
stat_dfs = {}

for year in YEARS:
    print(f"⏳ Fetching stats for {year} …")
    stat_dfs[year] = {}
    for stat_name, stat_code in stats.items():
        df = fetch_stat_details(stat_code,
                                year=year,
                                tournament_id=None)
        df['Year'] = year
        stat_dfs[year][stat_name] = df
    print(f"✔️  Done {year}")

⏳ Fetching stats for 2010 …
✔️  Done 2010
⏳ Fetching stats for 2011 …
✔️  Done 2011
⏳ Fetching stats for 2012 …
✔️  Done 2012
⏳ Fetching stats for 2013 …
✔️  Done 2013
⏳ Fetching stats for 2014 …
✔️  Done 2014
⏳ Fetching stats for 2015 …
✔️  Done 2015
⏳ Fetching stats for 2016 …
✔️  Done 2016
⏳ Fetching stats for 2017 …
✔️  Done 2017
⏳ Fetching stats for 2018 …
✔️  Done 2018
⏳ Fetching stats for 2019 …
✔️  Done 2019
⏳ Fetching stats for 2020 …
✔️  Done 2020
⏳ Fetching stats for 2021 …
✔️  Done 2021
⏳ Fetching stats for 2022 …
✔️  Done 2022
⏳ Fetching stats for 2023 …
✔️  Done 2023
⏳ Fetching stats for 2024 …
✔️  Done 2024


In [14]:
stat_dfs[2020]['Birdie Average']

,rank,playerId,playerName,Avg,# of Birdies,Total Rounds,Year
0,1,29221,Webb Simpson,4.67,243,52,2020
1,2,34363,Tyrrell Hatton,4.55,191,42,2020
2,2,33448,Justin Thomas,4.55,300,66,2020
3,4,47959,Bryson DeChambeau,4.42,274,62,2020
4,5,40026,Daniel Berger,4.38,280,64,2020
...,...,...,...,...,...,...,...
188,188,34256,Andrew Putnam,3.20,147,46,2020
189,190,49298,Kramer Hickok,3.18,156,49,2020
190,191,33120,Mark Anderson,3.04,143,47,2020
191,192,23623,Bo Van Pelt,2.98,128,43,2020
